In [ ]:
#For authentication from google drive and to link googe drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

# This cell contains all the references to libraries that are needed to train a convolutional neural network

from __future__ import print_function, division  
#To support Py2.0's print function  #To correct ambiguous division operator

import numpy as np   #for working with arrays
import random #to give a random number between 0 and 1
import os #The OS module in Python provides functions for creating and removing a directory (folder), fetching its contents, changing and identifying the current directory, etc.
import glob #To find files with a certain extension in a library. Eg: To find .jpg files

# https://opencv.org/
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2 #OpenCV with Numpy Support
import datetime
import pandas as pd #For data analysis
import time 
import h5py #To work with HDF5 format, which stores huge amounts of numerical data #https://www.h5py.org/ to store trained file
import csv #To handle CSV files (Excel)

# https://docs.scipy.org/doc/scipy-1.2.1/reference/generated/scipy.misc.imresize.html
from skimage.transform import resize

#from sklearn.cross_validation import KFold, train_test_split #To estimate skill of machine learning model #Split to Training and Testing, to avoid overfitting
from sklearn.metrics import log_loss, confusion_matrix #To calculate loss function #Matrix to visualize performance
from sklearn.utils import shuffle #shuffle data for better training

from PIL import Image, ImageChops, ImageOps  #To work with images #For image editing #For automatic filters on images
#PIL = Python Imaging Library

import matplotlib.pyplot as plt #To replicate plot function in matlab

from keras.layers import Dense, Dropout, Flatten, Lambda, ELU, Activation, BatchNormalization

from keras.optimizers import SGD, Adam, RMSprop

from keras import backend as K #To choose backend(Tensorflow,Theano,CTNK,etc)
from keras.callbacks import EarlyStopping, Callback #To stop training epochs if training stops improving, to prevent overfitting #To view staticstics during training
from keras.utils import np_utils #Embed NumPy with Keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array #For data augmentation(rotate,resize,shearing etc) #Load image as PIL #To make 3D NumPy array of img
from keras import optimizers #Algorithm to update weights
from keras.models import Sequential, model_from_json #For layer-by-layer model creation #A type of ANN model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Activation, Dropout, Flatten, Dense , Cropping2D #layers in CNN

In [ ]:
# Build a neural network in Keras
num_classes=10
# Function to resize image to 56x56
#def resize_image(image):
 #   import tensorflow as tf
 #   return tf.image.resize_images(image,[56,56])

# Function to resize image to 64x64
row, col, ch = 113, 113, 1

model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(row, col, ch)))

# Resise data within the neural network
#model.add(Lambda(resize_image))  #resize images to allow for easy computation

# CNN model - Building the model suggested in paper

model.add(Convolution2D(filters= 32, kernel_size =(5,5), strides= (2,2), padding='same', name='conv1')) #96
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2), name='pool1'))

model.add(Convolution2D(filters= 64, kernel_size =(3,3), strides= (1,1), padding='same', name='conv2'))  #256
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2), name='pool2'))

model.add(Convolution2D(filters= 128, kernel_size =(3,3), strides= (1,1), padding='same', name='conv3'))  #256
model.add(Activation('relu'))



model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(512, name='dense1'))  #1024
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256, name='dense2'))  #1024
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes,name='output'))
model.add(Activation('softmax'))  #softmax since output is within 50 classes

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
# use ImageDataGenerator to preprocess the data
from keras.preprocessing.image import ImageDataGenerator

# Augment the data that we have to create more training data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=0,
                                   shear_range = 0,
                                   zoom_range = 0)

test_datagen = ImageDataGenerator(rescale = 1./255)


batchsize = 32 #Default 32
# Prepare training data
# Select path, Input resolution, input resolution, Batch size, class number.
# The batch size is a number of samples processed before the model is updated. https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network
# class_mode selection - https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720
training_data = train_datagen.flow_from_directory('/content/drive/My Drive/cnn hw/Training',
                                                 target_size = (113, 113),color_mode= 'grayscale',
                                                 batch_size = batchsize,
                                                 class_mode = 'categorical')
# prepare test data
valid_data = test_datagen.flow_from_directory('/content/drive/My Drive/cnn hw/Testing',
                                            target_size = (113, 113),color_mode= 'grayscale',
                                            batch_size = batchsize,
                                            class_mode = 'categorical')

test_data = test_datagen.flow_from_directory('/content/drive/My Drive/cnn hw/Testing',
                                            target_size = (113, 113),color_mode= 'grayscale',
                                            batch_size = 1,
                                            class_mode = 'categorical')



Found 2946 images belonging to 10 classes.
Found 730 images belonging to 10 classes.
Found 730 images belonging to 10 classes.


In [ ]:
# finally start computation
# to improve the model accuracy you can increase the number of steps_per_epoch to e.g. 8000
# increase the number of epochs to 5-25
# increase the validation steps
# this parametters allow for the model to optimize
wandb.init(project="CNN hand v2")
model.fit_generator(training_data,
                         steps_per_epoch = (4000 / batchsize),  #real 4000
                         epochs = 30,
                         validation_data = valid_data,
                         validation_steps = 20,
                         callbacks=[WandbCallback()])

In [ ]:
# finally start computation
# to improve the model accuracy you can increase the number of steps_per_epoch to e.g. 8000
# increase the number of epochs to 5-25
# increase the validation steps
# this parametters allow for the model to optimize

model.fit_generator(training_data,
                         steps_per_epoch = (4000 / batchsize),  #real 4000
                         epochs = 30,
                         validation_data = valid_data,
                         validation_steps = 20
                         )

Epoch 1/30
124/125 [============================>.] - ETA: 10s - loss: 1.4280 - accuracy: 0.5061

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 15 could not be retrieved. It could be because a worker has died.
  UserWarning)


125/125 [==============================] - 1451s 12s/step - loss: 1.4258 - accuracy: 0.5076 - val_loss: 1.2878 - val_accuracy: 0.5781
Epoch 2/30
125/125 [==============================] - 40s 320ms/step - loss: 1.2311 - accuracy: 0.5509 - val_loss: 1.2184 - val_accuracy: 0.5426
Epoch 3/30
125/125 [==============================] - 6s 52ms/step - loss: 1.0902 - accuracy: 0.6060 - val_loss: 0.6914 - val_accuracy: 0.6104
Epoch 4/30
125/125 [==============================] - 6s 51ms/step - loss: 0.9062 - accuracy: 0.6655 - val_loss: 1.0855 - val_accuracy: 0.6782
Epoch 5/30
125/125 [==============================] - 6s 51ms/step - loss: 0.6996 - accuracy: 0.7469 - val_loss: 0.5168 - val_accuracy: 0.7666
Epoch 6/30
125/125 [==============================] - 6s 52ms/step - loss: 0.4697 - accuracy: 0.8330 - val_loss: 0.5911 - val_accuracy: 0.8044
Epoch 7/30
125/125 [==============================] - 6s 50ms/step - loss: 0.3768 - accuracy: 0.8708 - val_loss: 0.4591 - val_accuracy: 0.8170
Epoch 

In [ ]:
#To save training data
#To save whole data - classifier.save(filepath) , To save just weights without architecture - classifier.save_weights(filepath)
os.chdir("/content/drive/cnn hw/")
#classifier.save_model('location') , add architecture seperately
classifier.save('model1.h5py')

In [ ]:
#To load training data
from keras.models import load_model
classifier = load_model('/content/drive/cnn hw/model1.h5py')

In [ ]:
# for resize

import os
import glob
import  PIL
from PIL import Image
from random import sample 


images = []
path_to_files = os.path.join(r'C:\Users\ShaiZ\Desktop\hand\1', '*')
for filename in sorted(glob.glob(path_to_files)):
    image_name = filename.split('/')[-1]
    file, ext = os.path.splitext(image_name)
    im = Image.open(filename)
    cur_width = im.size[0]
    cur_height = im.size[1]

    # print(cur_width, cur_height)
    height_fac = 113 / cur_height

    new_width = int(cur_width * height_fac)
    size = new_width, 113

    imresize = im.resize((size), Image.ANTIALIAS)  # Resize so height = 113 while keeping aspect ratio
    now_width = imresize.size[0]
    now_height = imresize.size[1]
    # Generate crops of size 113x113 from this resized image and keep random 10% of crops

    avail_x_points = list(range(0, now_width - 113 ))# total x start points are from 0 to width -113

    # Pick random x%
    factor = 0.05
    pick_num = int(len(avail_x_points)*factor)
    
    random_startx = sample(avail_x_points,  pick_num)
    k=0
    for start in random_startx:
        imcrop = imresize.crop((start, 0, start+113, 113))
        t=str(k)
        imcrop.save(file +"new "+t +".png" , "PNG")
        k=k+1
        
